In [1]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 76.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 83.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4

In [92]:
import pandas as pd
import numpy as np

train = pd.read_csv('/kaggle/input/hack-museum/train.csv')
test = pd.read_csv('/kaggle/input/hack-museum/test.csv')
items = pd.read_csv('/kaggle/input/hack-museum/items.csv')

In [94]:
train

query_id item_id
0          i05  v02931
1          i05  s00310
2          i08  a00461
3          i08   s0044
4         i016  s00519
...        ...     ...
29815     s017    i569
29816     s017   i4148
29817     s017  i21616
29818     s017  i21617
29819     s017   i1505

[29820 rows x 2 columns]

In [95]:
items['text'] = items['title'].fillna('') + f'\n{"-" * 10}\n' + items['description'].fillna('')

In [96]:
import chromadb
from chromadb.utils import embedding_functions
import torch

In [12]:
ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="sergeyzh/rubert-mini-frida", device="cuda"
)

2025-11-12 09:04:05.919253: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762938246.056019      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762938246.096708      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/196 [00:00<?, ?B/s]

In [143]:
client = chromadb.Client()
collection = client.create_collection(name='items', embedding_function=ef)

In [145]:
from tqdm.auto import tqdm

for i in tqdm(range(0, len(items), 1000)):
    subset = items.iloc[i:i+1000]
    collection.add(
        documents=subset['text'].values.tolist(),
        ids=subset['item_id'].values.tolist(),
        metadatas=[{'modality': m, 'index': idx} for m, idx in zip(subset['modality'].values, subset['item_id'].values)]
    )

  0%|          | 0/8 [00:00<?, ?it/s]

In [24]:
test

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


id  relevant_ids
0       i01           NaN
1       i04           NaN
2       i11           NaN
3      i120           NaN
4      i123           NaN
..      ...           ...
299  a00936           NaN
300  a01127           NaN
301  a01128           NaN
302  a01129           NaN
303  a01140           NaN

[304 rows x 2 columns]

In [30]:
print(items['text'].iloc[1])

Cassiers H. Плакат. Всемирная выставка в Брюсселе. 1910
----------
Всемирная выставка в Брюсселе.


In [98]:
train

query_id item_id
0          i05  v02931
1          i05  s00310
2          i08  a00461
3          i08   s0044
4         i016  s00519
...        ...     ...
29815     s017    i569
29816     s017   i4148
29817     s017  i21616
29818     s017  i21617
29819     s017   i1505

[29820 rows x 2 columns]

In [109]:
i = 1
if len(train.loc[train['query_id'] == items['item_id'].iloc[i], 'item_id'].unique()) == 0:
    cond = {"modality": {"$ne": items['modality'].iloc[i]}}
else:
    cond = {"$and": [
                {"id": {"$nin": train.loc[train['query_id'] == items['item_id'].iloc[i], 'item_id'].unique()}},
                {"modality": {"$ne": items['modality'].iloc[i]}}
            ]
        }
result = collection.query(
    query_texts=[items['text'].iloc[i]],
    n_results=10,
    # where={"modality": {"$ne": items['modality'].iloc[i]}}
    where=cond
)
print("Query:")
print(items['text'].iloc[i])
print('Results:')
for r in result['documents'][0]:
    print(r, end='\n---------\n')

Query:
Cassiers H. Плакат. Всемирная выставка в Брюсселе. 1910
----------
Всемирная выставка в Брюсселе.
Results:
Анри Руссо. «Нападение ягуара на лошадь»
----------
<p>1910</p><p>холст, масло</p><p>90х116 см</p>
---------
Выставка  "Хендрик Керстенс. Паула"
----------
Серия женских фотографических портретов, напоминающих голландскую живопись XVII века.
---------
Irina Karasik. The State Museum of the Modern Western Art as a laboratory for the...
----------
Boris Vipper Memorial Conference - 2019 
v01111,video,Татьяна Юденкова. Сергей Третьяков и французская живопись,XLVIII Международная конференция «Випперовские чтения». Собрание Сергея Щукина: история
---------
Символист — для символистов | VK
----------
Лист Проекта обложки журнала «Весы» (для №4, 5, 6, 1904 г.) сейчас представлен на нашей выставке Избранных рисунков «От Дюрера до Матисса». Разнообразные рисунки Редона можно увидеть и в качестве заставок и концовок статей на страницах номеров 4 и 5 за 1904 г. — «всех художественных 

In [107]:
train.loc[train['query_id'] == items['item_id'].iloc[5], 'item_id'].unique()

array(['a00461', 's0044'], dtype=object)

In [104]:
train

query_id item_id
0          i05  v02931
1          i05  s00310
2          i08  a00461
3          i08   s0044
4         i016  s00519
...        ...     ...
29815     s017    i569
29816     s017   i4148
29817     s017  i21616
29818     s017  i21617
29819     s017   i1505

[29820 rows x 2 columns]

In [42]:
items.nunique()

item_id        7562
modality          4
title          6768
description    6053
extra          7402
text           6826
dtype: int64

In [40]:
test

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


id  relevant_ids
0       i01           NaN
1       i04           NaN
2       i11           NaN
3      i120           NaN
4      i123           NaN
..      ...           ...
299  a00936           NaN
300  a01127           NaN
301  a01128           NaN
302  a01129           NaN
303  a01140           NaN

[304 rows x 2 columns]

In [46]:
items.loc[items['item_id'] == 'i01', 'text'].iloc[0]

'Blessum B. Плакат. Добро пожаловать на землю викингов! 1920-е\n----------\nДобро пожаловать на землю викингов!'

In [58]:
test

id                                       relevant_ids
0       i01  v01555 s67 a00347 s60 v02137 v03339 v02249 v02...
1       i04  a00089 v02923 v01110 s118 v01471 v01104 v02242...
2       i11  a00916 s010 s53 v01552 s015 a00875 v01106 s013...
3      i120  a00205 a00726 a00455 s97 a00397 s85 a00199 a00...
4      i123  s00643 a00873 s00176 s0041 a00008 s97 a00089 s...
..      ...                                                ...
299  a00936  s00333 s0096 s00294 s00428 s18 s00223 s00224 s...
300  a01127  i1671 i3444 i4735 i26748 v01555 i1110 i20721 i...
301  a01128  s147 i16150 v01284 i26536 i19715 s132 i19350 i...
302  a01129  v01554 v00250 i3615 v01555 i16616 i15634 i1957...
303  a01140  v00250 i16133 i1224 i25818 i1225 i14827 i4523 ...

[304 rows x 2 columns]

In [150]:
preds = []
for idx in tqdm(test['id'].values):
    if len(items.loc[items['item_id'] == idx, 'text']) == 0:
        preds.append("")
        continue
    if len(train.loc[train['query_id'] == idx, 'item_id'].unique()) == 0:
        cond = {"modality": {"$ne": items.loc[items['item_id'] == idx, 'modality'].iloc[0]}}
    else:
        cond = {"$and": [
                    {"index": {"$nin": train.loc[train['query_id'] == idx, 'item_id'].unique().tolist()}},
                    {"modality": {"$ne": items.loc[items['item_id'] == idx, 'modality'].iloc[0]}}
                ]
            }
    result = collection.query(
        query_texts=[items.loc[items['item_id'] == idx, 'text'].iloc[0]],
        n_results=10,
        where=cond
    )
    preds.append(" ".join(result['ids'][0]))

  0%|          | 0/304 [00:00<?, ?it/s]

In [126]:
cond

{'$and': [{'id': {'$nin': array(['a00177', 'a00139', 'a00156'], dtype=object)}},
  {'modality': {'$ne': 'image'}}]}

In [151]:
print("\n\n".join(result['documents'][0]))

Симеонова С. Скульптурная группа. Человек в городе. 1970-1980-е гг.
----------
Скульптурная группа. Человек в городе.

Бари А. Скульптурная группа. Кентавр и лапиф. 1849
----------
Скульптурная группа. Кентавр и лапиф Тезей, сражающийся с кентавром Бианором бронза (темно-коричневая с зеленым)

Жилло К. Гравюра. Жизнь сатиров. Свадьба сатиров. 1710-е
----------
Жизнь сатиров. Свадьба сатиров

Гаварни П. Гравюра. Что это? Почтенные господа стали теперь связываться с людьми вашего сорта?.." Лист из серии "Парижский карнавал" 1841"
----------
Что это? Почтенные господа стали теперь связываться с людьми вашего сорта?.." Лист из серии "Парижский карнавал""

Ларш Ф.Р. Лампа в виде снопа колосьев с сидящими в нем фигурками детей, н. XX в.
----------
Лампа в виде снопа колосьев с сидящими в нем фигурками детей

Сметана Б. Ноты. Klavirni dilo (Svazek I). Prvni cykly. Bagately a impromtus (1844). Šest charakteristických skladeb, op.I (1848). Svatebni scény (1849). б/г
----------
Отдел музыкальной

In [134]:
preds

['v01555 s67 a00347 s60 v02137 v03339 v02249 v02247 a00187 v02055',
 'a00089 v02923 v01110 s118 v01471 v01104 v02242 v02556 v01875 v02931',
 'a00916 s010 s53 v01552 s015 a00875 v01106 s013 v01495 a00625',
 'a00205 a00726 a00455 s97 a00397 s85 a00199 a00394 a00691 a00724',
 's00643 a00873 s00176 s0041 a00008 s97 a00089 s0084 a00780 v01613',
 's00584 s60 a00894 v01328 v01205 v01269 v01311 s00835 s00836 v01316',
 'v01269 a00830 a00894 a00825 a00579 a00822 a00827 a00553 a00537 a00796',
 'a00878 a00827 s00778 s00779 a00780 a00431 v01328 s00869 v01318 a00579',
 'a00812 a00894 v01269 a00824 v01328 a00550 a00729 a00796 a00827 a00724',
 's00191 s00192 v01454 a00894 s00441 a00306 a00825 v02527 v02655 v01311',
 's60 v01269 v01373 v01328 v01311 s00835 s00836 v01316 v01205 v01369',
 'v01437 s60 v02146 v02152 v02148 a00229 v01269 v02151 v02154 v01316',
 'v02154 a00828 a00764 v02152 s60 v01369 v02148 v02146 v02153 v02151',
 'a00397 a00232 a00724 s60 v01373 a00378 v01205 v01316 v01328 a00162',
 's60 v

In [152]:
test['relevant_ids'] = preds

In [155]:
test.to_csv('sub_hack_rubert-mini-frida_title-and-desc-filter-train.csv', index=False)

In [154]:
test

id                                       relevant_ids
0       i01  v01555 s67 a00347 s60 v02137 v03339 v02249 v02...
1       i04  a00089 v02923 v01110 s118 v01471 v01104 v02242...
2       i11  a00916 s010 s53 v01552 s015 a00875 v01106 s013...
3      i120  a00205 a00726 a00455 s97 a00397 s85 a00199 a00...
4      i123  s00643 a00873 s00176 s0041 a00008 s97 a00089 s...
..      ...                                                ...
299  a00936  s00333 s0096 s00294 s00428 s18 s00223 s00224 s...
300  a01127  i1671 i3444 i4735 i26748 v01555 i1110 i20721 i...
301  a01128  s147 v01284 i26536 i19715 s132 i19350 i19719 i...
302  a01129  v01554 i14216 i3615 v01555 i16616 i15634 i1393...
303  a01140  i25818 i1225 i14827 i4523 i1888 i25925 i4948 s...

[304 rows x 2 columns]

In [140]:
train[train['query_id'] == 'a00005']

query_id item_id
1396   a00005  s00797
1397   a00005  i21841
1398   a00005  v01201
1399   a00005  i21711
1400   a00005  s00644
1401   a00005    i116

In [119]:
set(test['id']) & set(train['query_id'])

{'a00005',
 'a00006',
 'a00007',
 'a00011',
 'a00097',
 'a00109',
 'a00112',
 'a00116',
 'a00120',
 'a00121',
 'a00123',
 'a00124',
 'a00125',
 'a00126',
 'a00127',
 'a00128',
 'a00141',
 'a00142',
 'a00149',
 'a00158',
 'a00166',
 'a00171',
 'a00185',
 'a00195',
 'a00211',
 'a00250',
 'a00266',
 'a00281',
 'a00292',
 'a00298',
 'a00304',
 'a00305',
 'a00306',
 'a00307',
 'a00310',
 'a00312',
 'a00314',
 'a00315',
 'a00325',
 'a00341',
 'a00343',
 'a00352',
 'a00362',
 'a00366',
 'a00382',
 'a00392',
 'a00394',
 'a00398',
 'a00400',
 'a00402',
 'a00403',
 'a00447',
 'a00457',
 'a00473',
 'a00478',
 'a00479',
 'a00482',
 'a00487',
 'a00506',
 'a00516',
 'a00519',
 'a00553',
 'a00562',
 'a00575',
 'a00578',
 'a00589',
 'a00602',
 'a00625',
 'a00626',
 'a00627',
 'a00673',
 'a00676',
 'a00678',
 'a00685',
 'a00697',
 'a00715',
 'a00716',
 'a00740',
 'a00744',
 'a00745',
 'a00749',
 'a00798',
 'a00815',
 'a00821',
 'a00832',
 'a00835',
 'a00923',
 'a00926',
 'a00936',
 'a01127',
 'a01128',

In [153]:
test[test['id'] == 'a00005'].iloc[0, 1]

's00146 s0032 s00196 s00719 s10 s00657 s00652 i1249 s0034 s00525'

In [114]:
pd.read_csv('/kaggle/working/sub_hack_rubert-mini-frida_title-and-desc.csv')

id                                       relevant_ids
0       i01  v01555 s67 a00347 s60 v02137 v03339 v02249 v02...
1       i04  a00089 v02923 v01110 s118 v01471 v01104 v02242...
2       i11  a00916 s010 s53 v01552 s015 a00875 v01106 s013...
3      i120  a00205 a00726 a00455 s97 a00397 s85 a00199 a00...
4      i123  s00643 a00873 s00176 s0041 a00008 s97 a00089 s...
..      ...                                                ...
299  a00936  s00333 s0096 s00294 s00428 s18 s00223 s00224 s...
300  a01127  i1671 i3444 i4735 i26748 v01555 i1110 i20721 i...
301  a01128  s147 i16150 v01284 i26536 i19715 s132 i19350 i...
302  a01129  v01554 v00250 i3615 v01555 i16616 i15634 i1957...
303  a01140  v00250 i16133 i1224 i25818 i1225 i14827 i4523 ...

[304 rows x 2 columns]

In [91]:
items[items['item_id'] == 'a00366']

item_id modality                                              title  \
5582  a00366    audio  Джованни Франческо Барбьери, прозванный Гверчи...   

                                            description  \
5582  <p>Вера (лат. Fides)&nbsp;&mdash; одна из&nbsp...   

                                                  extra  \
5582  https://dgfi96202bcqs.cloudfront.net/b46a68c6-...   

                                                   text  
5582  Джованни Франческо Барбьери, прозванный Гверчи...

In [87]:
test

id                                       relevant_ids
0       i01  i23438 i24649 i18410 i05 i04 i3223 i18415 i184...
1       i04  i16530 i6033 i3121 i172 i25844 i4847 i20628 i1...
2       i11  i050 i6949 i6948 i25421 i703 i702 i6950 i17930...
3      i120  i23233 i20843 i16549 i4822 i19118 i15825 i1934...
4      i123  i7031 i25425 i20744 i1595 i1639 i557 i5526 i65...
..      ...                                                ...
299  a00936  a00369 a00366 a00367 a00365 a00368 a00937 a003...
300  a01127  i1671 i3444 i4735 i26748 v01555 i1110 i20721 i...
301  a01128  s147 i16150 v01284 i26536 i19715 s132 i19350 i...
302  a01129  a01140 v01554 a00054 v00250 i3615 a00153 v0155...
303  a01140  a01129 v00250 a01127 i16133 i1224 i25818 i1225...

[304 rows x 2 columns]